In [203]:
import pandas as pd
import datetime

prec = pd.read_csv("Daily Precipitation.csv")
soil = pd.read_csv("Daily Soil Moisture.csv")
ed = pd.read_csv("Eight Day NDVI.csv")
prodq = pd.read_csv("Production Quantity.csv")
preds = pd.read_csv("predicted_production_qty.csv")
dates = preds.iloc[:2].copy()


In [204]:
def clean_date(data, date_col=["start_date"]):
    """This function cleans and parses date columns to the datetime type"""
    for datum in data:
        # Drop column "end_date" since it does not match with production quantity dataset
        datum.drop(columns="end_date", inplace=True)
        for col in date_col:
            # Keep the first 8 charcters of col value
            datum[col] = datum[col].str.replace("-", "").str[:8]
            # Parse col to datetime type
            datum[col] = pd.to_datetime(datum[col], format="%Y%m%d")


clean_date([prec, soil, ed])


In ed, the ndvi score is measured every 8 days. Thus, I will add more dates to fill in the gap between the start_date and end_date (7 more dates) and adjust the score in the dates by a linear difference between the previous and the current date by the formula 
$$
ndvi_{t+1} = ndvi_{t} + \frac{ndvi_{t+8} - ndvi_{t}} {t + 8 - t} * (t + 1 - t)
$$
<br>
For example, the ndvi on start_date of 2014-01-01 and 2014-01-09 are 0.701431 and 0.745149. Then, the ndvi on the start_date of 2014-01-04 is<br>
0.701431 + (0.745149 - 0.701431) / 8 * 3<br>
This method is inspired by back propagation in Deep Learning



In [205]:
def unique_date(df, date_col):
    """This function returns a list of dates that were between newest and oldest dates in date_col values"""
    dates = pd.date_range(start=min(df[date_col]), end=max(df[date_col])).to_list()
    for date in dates:
        if date in df[date_col].values:
            dates.remove(date)
    return dates


def closest_date(date, df, date_col):
    """This function returns the index of the closest date in df[date_col] that was before the date"""
    index = df[date_col].searchsorted(date)
    return index


def impute_ndvi(date, df, index, date_col):
    """This function applies the formula above. Also, it returns the value of region_id"""
    pre_nvdi_value = df.iloc[index, -2]
    if index + 1 < len(df):
        post_nvdi_value = df.iloc[index + 1, -2]
    else:
        post_nvdi_value = pre_nvdi_value
    # Time difference between t and t-1 in terms of days
    time_delta = (df.loc[index, date_col] - date).days
    result = pre_nvdi_value + (post_nvdi_value - pre_nvdi_value) * time_delta / 8
    return result, df.loc[index, "region_id"]


ed_dates = unique_date(ed, "start_date")
new = pd.DataFrame()
for date in ed_dates:
    index = closest_date(date, ed, "start_date")
    new = new.append(
        {
            "start_date": date,
            "ndvi": impute_ndvi(date, ed, index, "start_date")[0],
            "region_id": impute_ndvi(date, ed, index, "start_date")[1],
        },
        ignore_index=True,
    )
ed = pd.concat([ed, new], join="inner")


C:\Users\dtngh\AppData\Local\Temp\ipykernel_2816\1056227833.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ed = new.append(


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
# def merging(data, columns=['start_date', 'region_id']):
#     '''This function merges all dataframes on columns'''
#     merged_data = data[0].copy()
#     for datum in data:
#         merged_data = pd.merge(merged_data, datum, on=columns)
#     return merged_data.drop_duplicates()


In [ ]:
# merged_data = merging([prec, soil, ed])
# print(len(merged_data))


In [ ]:
# merged_data
